In [ ]:
import numpy as np
from skimage import io, filters, util

In [ ]:
# Load the image
gray_image = io.imread('dataset/flooded/0.jpg', as_gray=True)

# Determine the threshold value using Otsu's method
threshold_value = filters.threshold_otsu(gray_image)

# Apply the threshold to the image
thresholded_image = gray_image > threshold_value

# Convert the boolean array to an 8-bit unsigned integer array
thresholded_image = util.img_as_ubyte(thresholded_image)

# Display the original and thresholded images
io.imshow(gray_image)
io.show()
io.imshow(thresholded_image)
io.show()

In [ ]:
# Load the grayscale image
gray_image = io.imread('dataset/flooded/0.jpg', as_gray=True)

# Determine the threshold value using Otsu's method
threshold_value = filters.threshold_otsu(gray_image)

# Apply the threshold to the grayscale image to create a mask
mask = gray_image > threshold_value

# Load the color image
color_image = io.imread('dataset/flooded/0.jpg')

# Apply the mask to the color image
masked_image = color_image.copy()
masked_image[~mask] = 0

# Display the original and masked images
io.imshow(color_image)
io.show()
io.imshow(masked_image)
io.show()

In [ ]:
from skimage import io, filters, color, util

# Load the color image
image = io.imread('dataset/flooded/0.jpg')

# Convert the image from RGB to HSV color space
hsv_image = color.rgb2hsv(image)

# Extract the saturation channel from the HSV image
saturation_channel = hsv_image[:, :, 1]

# Determine the threshold value using Otsu's method
threshold_value = filters.threshold_otsu(saturation_channel)

# Apply the threshold to the saturation channel to create a mask
mask = saturation_channel > threshold_value

# Apply the mask to the color image
masked_image = image.copy()
masked_image[mask] = 0

# Display the original and masked images
io.imshow(image)
io.show()
io.imshow(masked_image)
io.show()

In [ ]:
from sklearn.cluster import KMeans
from skimage import io

# Load the color image
image = io.imread('dataset/flooded/0.jpg')

# Reshape the image into a 2D array of pixels
pixels = image.reshape(-1, 3)

# Create a KMeans model with 2 clusters (water and non-water)
kmeans = KMeans(n_clusters=2)

# Fit the KMeans model to the pixel data
kmeans.fit(pixels)

# Predict the cluster labels for each pixel
labels = kmeans.predict(pixels)

# Reshape the labels back into the shape of the original image
labels = labels.reshape(image.shape[:2])

# Create a mask for the water pixels
mask = labels == 0

# Apply the mask to the color image
masked_image = image.copy()

In [ ]:
from skimage import io, filters, color

# Load the grayscale image
image = io.imread('dataset/flooded/0.jpg', as_gray=True)

# Determine the threshold values using multi-Otsu thresholding
threshold_values = filters.threshold_multiotsu(image, classes=5)

# Apply the thresholds to the image to create a multi-level mask
mask = np.digitize(image, bins=threshold_values)

# Display the original and masked images
io.imshow(image)
io.show()
io.imshow(mask)
io.show()

In [ ]:
from skimage import io, filters, color, util
import matplotlib.pyplot as plt

n_classes = 2

# Load the grayscale image
gray_image = io.imread('dataset/flooded/0.jpg', as_gray=True)

# Determine the threshold values using multi-Otsu thresholding
threshold_values = filters.threshold_multiotsu(gray_image, classes=n_classes)

# Apply the thresholds to the grayscale image to create a multi-level mask
mask = np.digitize(gray_image, bins=threshold_values)

# Load the color image
color_image = io.imread('dataset/flooded/0.jpg')

# Create a masked image for each class
masked_images = []
for i in range(n_classes):
    # Create a mask for the current class
    class_mask = mask == i

    # Apply the mask to the color image
    masked_image = color_image.copy()
    masked_image[~class_mask] = 0

    # Convert the masked image to an 8-bit unsigned integer array
    masked_image = util.img_as_ubyte(masked_image)

    # Add the masked image to the list of masked images
    masked_images.append(masked_image)

# Display the original and masked images
fig, axes = plt.subplots(1, n_classes + 1)
fig.set_size_inches(12, 10)
axes[0].imshow(color_image)
axes[0].set_title('Original Image')
for i in range(n_classes):
    axes[i+1].imshow(masked_images[i])
    axes[i+1].set_title(f'Mask {i+1}')
plt.show()

In [ ]:
# Compare between the mean of the pixel values of the original image and the masked images
print(f'Original image mean = {color_image.mean():.2f}')
for i in range(n_classes):
    print(f'Mask {i+1} mean = {masked_images[i].mean():.2f}')
    print(f'Mask {i+1} std = {masked_images[i].std():.2f}')



In [ ]:
import numpy as np
import cv2
from scipy import ndimage
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
import matplotlib.pyplot as plt

# Load the image
img = cv2.imread('dataset/flooded/0.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to the image
gray = cv2.GaussianBlur(gray, (3, 3), 0)

# Find the threshold values using Otsu's method
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Find the distance transform of the thresholded image
dist_transform = ndimage.distance_transform_edt(thresh)

# Find the local maxima of the distance transform
local_maxi = peak_local_max(dist_transform, indices=False, footprint=np.ones((3, 3)), labels=thresh)

# Label the local maxima
markers = ndimage.label(local_maxi)[0]

# Apply watershed segmentation to the image
labels = watershed(-dist_transform, markers, mask=thresh)

In [ ]:
# Display the original image
plt.imshow(img)
plt.title('Original Image')
plt.show()

# Display the segmented image
plt.imshow(labels, cmap='nipy_spectral')
plt.title('Segmented Image')
plt.show()

In [ ]:
# Use the black pixels in the labels array as a mask to separate the background from the foreground
mask = labels != 0

# Create a masked image
masked_image = img.copy()
masked_image[mask] = 0

# Display the masked image
plt.imshow(masked_image)
plt.title('Masked Image')
plt.show()

In [ ]:
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth
from PIL import Image

# Load the original image
originImg = Image.open('dataset/flooded/0.jpg')

# Convert the image to a NumPy array
originImg = np.array(originImg)

# transform the image to 224 x 224 x 3
originImg = cv2.resize(originImg, (448, 448))

# Get the shape of the original image
originShape = originImg.shape

In [ ]:
# Convert the image into an array of dimension [number of pixels in originImage, 3]
# based on RGB intensities
flatImg = np.reshape(originImg, [-1, 3])

# Estimate bandwidth for the mean shift algorithm
bandwidth = estimate_bandwidth(flatImg, quantile=0.1)

ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)

# Perform mean shift on flatImg
ms.fit(flatImg)

In [ ]:
# Get the labels for each point in the image
labels = ms.labels_

# Get the cluster centers
cluster_centers = ms.cluster_centers_

# Find and display the number of clusters
labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)
print("number of estimated clusters : %d" % n_clusters_)

segmentedImg = cluster_centers[np.reshape(labels, originShape[:2])]

segmentedImg = Image.fromarray(segmentedImg.astype(np.uint8))
segmentedImg.show()